In [48]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
data  = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/saarthi/train_data.csv")
data.head()

,path,transcription,action,object,location
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none


In [11]:
data = data.iloc[0:,1:]
data.head()

,transcription,action,object,location
0,Turn on the kitchen lights,activate,lights,kitchen
1,Turn up the temperature,increase,heat,none
2,OK now switch the main language to Chinese,change language,Chinese,none
3,Turn down the bathroom temperature,decrease,heat,washroom
4,Change the language,change language,none,none


In [28]:

ps = PorterStemmer()
corpus = []
for i in range(0, len(data)):
    
    review = re.sub('[^a-zA-Z]', ' ', data['transcription'][i])
    review = review.lower()
    review = review.split()
    
    transcript = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    transcript = ' '.join(review)
    corpus.append(review)

In [35]:

lem = WordNetLemmatizer()
corpus = []
for i in range(0, len(data)):
    
    transcript = re.sub('[^a-zA-Z]', ' ', data['transcription'][i])
    transcript = transcript.lower()
    transcript = transcript.split()
    
    transcript = [lem.lemmatize(word) for word in transcript if not word in stopwords.words('english')]
    transcript = ' '.join(transcript)
    #corpus.append(transcript)
    data['trans'][i] = transcript 
  

In [43]:
data.head()

,transcription,action,object,location,trans
0,Turn on the kitchen lights,activate,lights,kitchen,turn kitchen light
1,Turn up the temperature,increase,heat,none,turn temperature
2,OK now switch the main language to Chinese,change language,Chinese,none,ok switch main language chinese
3,Turn down the bathroom temperature,decrease,heat,washroom,turn bathroom temperature
4,Change the language,change language,none,none,change language


In [45]:
### Vocabulary size
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in data.trans] 
onehot_repr[0]

[1131, 3042, 4368]

In [46]:
sent_length=15
embedded_trans=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_trans[0])

[   0    0    0    0    0    0    0    0    0    0    0    0 1131 3042
 4368]


In [57]:
le  = LabelEncoder()
categ = ['action','object','location']

# Encode Categorical Columns
le = LabelEncoder()
data[['act','obj','loca']] = data[categ].apply(le.fit_transform)

In [58]:
data.head()

,transcription,action,object,location,trans,act,obj,loca
0,Turn on the kitchen lights,activate,lights,kitchen,turn kitchen light,0,7,1
1,Turn up the temperature,increase,heat,none,turn temperature,5,4,2
2,OK now switch the main language to Chinese,change language,Chinese,none,ok switch main language chinese,2,0,2
3,Turn down the bathroom temperature,decrease,heat,washroom,turn bathroom temperature,4,4,3
4,Change the language,change language,none,none,change language,2,10,2


In [62]:
x = embedded_trans
y = data.iloc[:,5:]

In [80]:
x.shape,y.shape

((11566, 15), (11566, 3))

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [79]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(3,activation='sigmoid'))
model.compile(loss='CategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 256,703
Trainable params: 256,703
Non-trainable params: 0
_________________________________________________________________
None


In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [82]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
122/122 [==============================] - 6s 31ms/step - loss: 33.5288 - accuracy: 0.6327 - val_loss: 38.8462 - val_accuracy: 0.6408
Epoch 2/10
122/122 [==============================] - 3s 27ms/step - loss: 39.5898 - accuracy: 0.6330 - val_loss: 40.0118 - val_accuracy: 0.6408
Epoch 3/10
122/122 [==============================] - 3s 27ms/step - loss: 40.1524 - accuracy: 0.6330 - val_loss: 39.9488 - val_accuracy: 0.6408
Epoch 4/10
122/122 [==============================] - 3s 26ms/step - loss: 40.3674 - accuracy: 0.6330 - val_loss: 40.1146 - val_accuracy: 0.6408
Epoch 5/10
122/122 [==============================] - 3s 26ms/step - loss: 40.4996 - accuracy: 0.6330 - val_loss: 40.0157 - val_accuracy: 0.6408
Epoch 6/10
122/122 [==============================] - 3s 27ms/step - loss: 40.4702 - accuracy: 0.6330 - val_loss: 40.0302 - val_accuracy: 0.6408
Epoch 7/10
122/122 [==============================] - 3s 27ms/step - loss: 40.3600 - accuracy: 0.6330 - val_loss: 40.0716 - val_ac

In [91]:
#train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
test_predict
#test_predict = pd.DataFrame(test_predict)
#train_predict=le.inverse_transform(train_predict)
test_predict = test_predict.apply(le.inverse_transform)



#from sklearn.metrics import mean_squared_error
#math.sqrt(mean_squared_error(y_train,train_predict))
#math.sqrt(mean_squared_error(ytest,test_predict))

ValueError: ignored